In [4]:
import os
import pandas as pd
import numpy as np
import datetime
import scipy
from scipy.stats import skew, kurtosis

In [44]:
os.listdir('../data/clean_tac')

base_path = '../data/clean_tac'

frame = pd.read_csv('../data/all_accelerometer_data_pids_13.csv')



In [45]:
# # frame.groupby(by=["pid"])
# # frame.groupby(["pid"]).count()

# frame
# SA0297
def get_time_ignore_second(x):
    t = datetime.datetime.fromtimestamp(x/1000.0)
    t = t.replace(microsecond = 0)
    t = t.replace(second = int(t.second / 10))
    return t.timestamp()

# frame_temp = frame[frame.pid == "SA0297"]
frame['window10'] = frame['time'].apply(get_time_ignore_second)
frame_temp = frame.drop(columns="time")
# frame_temp.head


<bound method NDFrame.head of              pid         x         y         z      window10
0         JB3156  0.000000  0.000000  0.000000  0.000000e+00
1         CC6740  0.000000  0.000000  0.000000  0.000000e+00
2         SA0297  0.075800  0.027300 -0.010200  1.493734e+09
3         SA0297 -0.035900  0.079400  0.003700  1.493734e+09
4         SA0297 -0.242700 -0.086100 -0.016300  1.493734e+09
...          ...       ...       ...       ...           ...
14057562  CC6740 -0.133956  0.124726 -0.010736  1.493829e+09
14057563  CC6740 -0.100764  0.180872  0.046449  1.493829e+09
14057564  CC6740 -0.131853  0.195934  0.181088  1.493829e+09
14057565  CC6740 -0.149704  0.194482  0.202393  1.493829e+09
14057566  CC6740 -0.107288  0.153548  0.168595  1.493829e+09

[14057567 rows x 5 columns]>

In [54]:
# print(frame_temp.groupby("window10").count())
pids = frame_temp.pid.unique()
frame_temp


# print(frame_temp.size)

,pid,x,y,z,window10
0,JB3156,0.000000,0.000000,0.000000,0.000000e+00
1,CC6740,0.000000,0.000000,0.000000,0.000000e+00
2,SA0297,0.075800,0.027300,-0.010200,1.493734e+09
3,SA0297,-0.035900,0.079400,0.003700,1.493734e+09
4,SA0297,-0.242700,-0.086100,-0.016300,1.493734e+09
...,...,...,...,...,...
14057562,CC6740,-0.133956,0.124726,-0.010736,1.493829e+09
14057563,CC6740,-0.100764,0.180872,0.046449,1.493829e+09
14057564,CC6740,-0.131853,0.195934,0.181088,1.493829e+09
14057565,CC6740,-0.149704,0.194482,0.202393,1.493829e+09


In [105]:
# TODO: Make n = 10 after either removing one record which has 7 records for a second or by adding 3 dummy values to it (latter is better)
# frame_temp.groupby([ "pid", "window10"]).count().describe()
# We are sampling with replacement, which should be okay since it is within a second
frame_temp2 = frame_temp.groupby([ "pid", "window10"]).sample(n = 20, replace=True)


In [74]:

# frame_temp2
# frame_temp2.groupby([ "pid", "window10"]).describe()

,pid,x,y,z,window10
47798,BK7610,-0.0163,-0.0280,0.0027,1.493736e+09
47348,BK7610,-0.0121,-0.0107,0.0280,1.493736e+09
47484,BK7610,-0.0023,0.0300,0.1079,1.493736e+09
47670,BK7610,0.0137,-0.0014,-0.0155,1.493736e+09
47304,BK7610,-0.0052,-0.0018,-0.0083,1.493736e+09
...,...,...,...,...,...
8652227,SF3079,0.0685,0.0441,-0.0442,1.493791e+09
8652105,SF3079,0.0071,0.0139,0.0183,1.493791e+09
8652382,SF3079,0.0619,0.1040,0.1627,1.493791e+09
8652100,SF3079,0.0057,0.0135,0.0006,1.493791e+09


In [75]:
# frame_temp

data = frame_temp2.to_numpy()

# dataset = []
# np.ndarray(dataset)

In [130]:
# x_sliding_window = np.lib.stride_tricks.sliding_window_view(data[:, 1], window_shape = 10)
# x_sliding_window.shape
final = []
for pid in pids:
  temptemp = frame_temp2.loc[frame_temp2['pid'] == pid]
  times = temptemp.window10.unique()
  final_temp =[]
  for time in times:
    # x = np.lib.stride_tricks.sliding_window_view(frame_temp2[frame_temp2.pid == pid and frame_temp2.window10 == time], window_shape = 10)
    temptemptemp = temptemp.loc[ (temptemp['window10'] == time)]
    # TODO: Create x y z sliding windows
    # x = np.lib.stride_tricks.sliding_window_view(temptemp["x"], window_shape = 200)
    # y = np.lib.stride_tricks.sliding_window_view(temptemp["y"], window_shape = 200)
    # z = np.lib.stride_tricks.sliding_window_view(temptemp["z"], window_shape = 200)
    a = np.vstack((temptemptemp["x"], temptemptemp["y"], temptemptemp["z"]))
    # print(temptemp.groupby("window10").count())
    # print(a)
    final_temp.append(a)
    # break
  final.append(np.array(final_temp))
  # print(final)
  
  # break
# print(np.array(final,dtype=object).shape)


In [131]:
print(np.array(final,dtype=object).shape)

(13,)
